In [1]:
import mne
from patient import Patient

/home/garkots/.conda/envs/garkots/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [15]:
import mne
import numpy as np
import pandas as pd
from tqdm import tqdm
import random

import matplotlib.pyplot as plt

from pingouin import partial_corr, corr
from sympy.utilities.iterables import multiset_permutations

from utils import translate, get_average, read_annotations, no_channels_read_annotations, save_to_json
from errors import OutOfSizeError

class Patient:


    def __init__(self, edf_file,
                 annotations,
                 path_to_visual_outputs,
                 path_to_ica_matrix,
                 path_to_results,
                 window_length,
                 ignore_channels,
                 precomputed_ica,
                 chosen_components,
                 control_experiment):

        self.SEED = 97

        self.chosen_components = chosen_components
        self.control_experiment = control_experiment

        print("CHOSEN COMPONENTS", self.chosen_components)

        self.window_to_detect = window_length
        self.precomputed_ica = precomputed_ica

        self.raw = mne.io.read_raw_edf(edf_file)
        self.raw_data = self.raw.get_data()
        # .filter(l_freq=1., h_freq=None)
        self.fs = self.raw.info['sfreq']
        self.ch_names = self.raw.ch_names

        if not ignore_channels:
            self.annotations = read_annotations(annotations)
        else:
            self.annotations = no_channels_read_annotations(annotations,
                                                            fs = self.fs,
                                                            window=window_length,
                                                            max_length=self.raw_data.shape[1])

            if control_experiment:
                key_random_annotations = [i for i in self.annotations.keys()][0]
                len_values = len(self.annotations[key_random_annotations])
                new_vals = [random.randrange(window_length, self.raw_data.shape[1]-window_length) for i in range(len_values)]
                self.annotations[key_random_annotations] = new_vals

        self.path_to_visual_outputs = path_to_visual_outputs
        self.path_to_results = path_to_results
        self.path_to_ica_matrix = path_to_ica_matrix



    def quantify_annotations(self):

        for k in self.annotations:
            print('Channel ', k, 'with ', len(self.annotations[k]), 'markings')

        self.all_markings = set()
        for timestamps in self.annotations:
            self.all_markings = self.all_markings.union(set(self.annotations[timestamps]))
        print("Total number of markings", len(self.all_markings))


        general_info = {'fs':self.fs,
                        'number of markings':len(self.all_markings)}

        save_to_json(self.path_to_results+'/general_info.json', general_info)

    def run_ica(self, variability_explained=0.95):
        '''
        Function that runs the ICA algorithm on the raw data. Saves the decomposed sources into the variable self.ica_sources_data
        :param variability_explained: float
        '''
        if self.precomputed_ica:
            self.ica_sources_data = np.load(self.path_to_ica_matrix+'/ica_matrix.npy')

            self.ica_sources_ch_names = []
            with open(self.path_to_ica_matrix+"/component_names.txt", "r") as f:
                for line in f:
                    self.ica_sources_ch_names.append(line.strip())

        else:
            ica = mne.preprocessing.ICA(n_components=variability_explained, random_state=self.SEED)
            ica.fit(self.raw)
            self.ica = ica
            ica_sources = ica.get_sources(self.raw)
            self.ica_sources_ch_names = ica_sources.ch_names
            self.ica_sources_data = ica_sources.get_data()
            self.mixing_matrix_ = ica.mixing_matrix_

            np.save(self.path_to_ica_matrix+'/ica_matrix.npy', self.ica_sources_data)

            with open(self.path_to_ica_matrix+"/component_names.txt", "w") as f:
                for s in self.ica_sources_ch_names:
                    f.write(str(s) + "\n")

            self.precomputed_ica = True

    def no_channels_during_seizures_states(self):
        '''
        The function is used to derive and save the patient's state during IEDs into  a dictionary: self.per_component_state_during_seizures
        Function stores the state of all channels and componets before and after IED in scope of a window
        Created for analysis of channels with no prior text-processing
        :return:
        '''

        per_channel_ieds, per_component_state_during_seizures = {}, {}
        for i, channel in enumerate(self.ch_names):

            data = self.raw_data[i]
            for timestamp in self.all_markings:
                pointer = timestamp

                ranged_data = data[pointer - self.window_to_detect:pointer + self.window_to_detect]

                if len(ranged_data) != self.window_to_detect * 2:
                    raise OutOfSizeError(pointer)

                if channel not in per_channel_ieds.keys():
                    per_channel_ieds[channel] = [ranged_data]
                else:
                    per_channel_ieds[channel].append(ranged_data)

        self.per_channel_ieds = per_channel_ieds
        if self.precomputed_ica:

            for i, ch in enumerate(self.ica_sources_ch_names):

                data = self.ica_sources_data[i]

                for timestamp in self.all_markings:

                    pointer = timestamp

                    ranged_data = data[pointer - self.window_to_detect:pointer + self.window_to_detect]

                    if ch not in per_component_state_during_seizures.keys():
                        per_component_state_during_seizures[ch] = [ranged_data]
                    else:
                        per_component_state_during_seizures[ch].append(ranged_data)

            self.per_channel_ieds, self.per_component_state_during_seizures = per_channel_ieds, per_component_state_during_seizures

    # def during_ieds_states(self):
    #     '''
    #     The function is used to derive and save the patient's state during IEDs into  a dictionary: self.per_component_state_during_seizures
    #     :return:
    #     '''
    #     per_channel_ieds, per_component_state_during_seizures = {}, {}
    #
    #     print("Text-processed channels:")
    #     # Some channels are not text-processed conviniently and need the following text-processing
    #     for channel in self.annotations:
    #         print(channel)
    #
    #         try:
    #             for ch_raw in self.ch_names:
    #                 if channel.upper() in ch_raw and channel not in per_channel_ieds.keys():
    #                     channel = ch_raw
    #
    #             data = self.raw_data[self.ch_names.index(channel.upper())]
    #         except:
    #             continue
    #
    #         for timestamp in self.annotations[channel[:3]]:
    #
    #             pointer = int(timestamp * self.fs)
    #
    #             ranged_data = data[pointer - self.window_to_detect:pointer + self.window_to_detect]
    #
    #             if channel not in per_channel_ieds.keys():
    #                 per_channel_ieds[channel] = [ranged_data]
    #             else:
    #                 per_channel_ieds[channel].append(ranged_data)
    #
    #     for i, ch in enumerate(self.ica_sources_ch_names):
    #
    #         data = self.ica_sources_data[i]
    #
    #         for timestamp in self.all_markings:
    #             pointer = int(timestamp * self.fs)
    #
    #             ranged_data = data[pointer - self.window_to_detect:pointer + self.window_to_detect]
    #
    #             if ch not in per_component_state_during_seizures.keys():
    #                 per_component_state_during_seizures[ch] = [ranged_data]
    #             else:
    #                 per_component_state_during_seizures[ch].append(ranged_data)
    #
    #     self.per_channel_ieds, self.per_component_state_during_seizures = per_channel_ieds, per_component_state_during_seizures

    def normalize_components(self):
        '''
        The function is used to normalize the components, s.t. while visualizing they will be comparable to the signal on the electrodes during IEDs.
        '''
        normalized_per_component_states = {}
        leftMin, leftMax = np.min(self.ica_sources_data), np.max(self.ica_sources_data)
        rightMin, rightMax = np.min(self.raw_data), np.max(self.raw_data)

        for c in self.per_component_state_during_seizures:
            result = []

            for seizure in self.per_component_state_during_seizures[c]:
                normalized_seizure = []
                for el in seizure:
                    normalized_seizure.append(translate(el, leftMin, leftMax, rightMin, rightMax))
                result.append(normalized_seizure)
            normalized_per_component_states[c] = np.array(result)

        self.normalized_per_component_states = normalized_per_component_states

    def plot_all_components_during_seizures(self, savefigures=False):
        '''
        The function is used to plot the states of the components during all the IEDs.
        '''

        for channel in self.per_channel_ieds:

            average_signal, signal_stdev = get_average(self.per_channel_ieds[channel])

            duration_timestamp = [i - self.window_to_detect for i in range(len(average_signal))]

            fig = plt.figure(figsize=(24, 10))

            plt.plot(duration_timestamp, average_signal, lw=3, label='pattern')
            lower_bound = average_signal - signal_stdev
            upper_bound = average_signal + signal_stdev
            plt.fill_between(duration_timestamp, lower_bound, upper_bound, facecolor='lightgoldenrodyellow', alpha=0.5,
                             label='1 sigma range')

            plt.title("Average state during IEDs on channel " + str(channel), fontsize=45)

            for component in self.normalized_per_component_states:
                average_component, component_stdev = get_average(self.normalized_per_component_states[component])

                plt.plot(duration_timestamp, average_component, lw=1, label=component)

            plt.title("Average state during IEDs on channel " + str(channel), fontsize=45)
            plt.legend(prop={'size': 7})
            plt.xlabel('Time relative to an IED', fontsize=40)
            plt.ylabel('Signal, mV', fontsize=40)

            plt.xticks(fontsize = 30)
            plt.yticks(fontsize=30)

            if savefigures:
                plt.savefig(self.path_to_visual_outputs+'/all_components/ch_'+str(channel)+'.jpg')
            plt.close()

    def plot_selected_components_during_seizures(self, potentially_significant, savefigures = False):
        '''
        The function plots channels stated as well as states on selected components during IEDs.
        :param potentially_significant: list(str)
        '''
        for channel in self.per_channel_ieds:

            average_signal, signal_stdev = get_average(self.per_channel_ieds[channel])
            duration_timestamp = [i for i in range(len(average_signal))]

            fig = plt.figure(figsize=(24, 10))

            plt.plot(duration_timestamp, average_signal, lw=3, label='pattern')
            lower_bound = average_signal - signal_stdev
            upper_bound = average_signal + signal_stdev
            plt.fill_between(duration_timestamp, lower_bound, upper_bound, facecolor='lightgoldenrodyellow', alpha=0.5,
                             label='1 sigma range')

            for component in self.normalized_per_component_states:

                if component in potentially_significant:
                    average_component, component_stdev = get_average(self.normalized_per_component_states[component])
                    plt.plot(duration_timestamp, average_component, lw=1, label=component)

            plt.title("Average state during IEDs on channel " + str(channel), fontsize=45)
            plt.legend()
            plt.xlabel('Time relative to an IED', fontsize=40)
            plt.ylabel('Signal, mV', fontsize=40)

            plt.xticks(fontsize = 30)
            plt.yticks(fontsize=30)
            plt.legend(prop={'size': 30})

            if savefigures:
                plt.savefig(self.path_to_visual_outputs+'/significant_components/ch_'+str(channel)+'.jpg')
            plt.close()


    def plot_components_during_seizures_nochannels(self, potentially_significant=False, savefigures = False):
        '''
        The function plots only selected components during IEDs.
        :param potentially_significant: list(str)
        '''
        if potentially_significant:
            fig = plt.figure(figsize=(24, 10))
            plt.title('Average components state during seizures', fontsize=23)
            for component in self.per_component_state_during_seizures:
                if component in potentially_significant:
                    average_component, component_stdev = get_average(
                        self.normalized_per_component_states[component])
                    duration_timestamp = [i for i in range(len(average_component))]
                    plt.plot(duration_timestamp, average_component, lw=1, label=component)

            if savefigures:
                plt.savefig(self.path_to_visual_outputs + '/potentially_significant_during_seizures.jpg')
            plt.close()

        else:
            fig = plt.figure(figsize=(24, 10))
            plt.title('Average components state during seizures', fontsize=23)
            for component in self.per_component_state_during_seizures:
                average_component, component_stdev = get_average(self.normalized_per_component_states[component])
                duration_timestamp = [i for i in range(len(average_component))]
                plt.plot(duration_timestamp, average_component, lw=1, label=component)

            if savefigures:
                plt.savefig(self.path_to_visual_outputs + '/all_components_during_seizures.jpg')
            plt.close()

    def plot_on_channel(self, ch_index_list, savefigures = False):

        fig = plt.figure(figsize=(24, 10))

        for i, channel in enumerate(self.per_channel_ieds):

            if i in ch_index_list or channel in ch_index_list:
                print("Channel ", channel)

                average_signal, signal_stdev = get_average(self.per_channel_ieds[channel])

                duration_timestamp = [i - self.window_to_detect for i in range(len(average_signal))]

                fig = plt.figure(figsize=(24, 10))

                plt.plot(duration_timestamp, average_signal, lw=3, label='pattern')
                lower_bound = average_signal - signal_stdev
                upper_bound = average_signal + signal_stdev
                plt.fill_between(duration_timestamp, lower_bound, upper_bound, facecolor='lightyellow', alpha=0.5,
                                 label='1 sigma range')

                plt.title("Channel " + str(channel), fontsize=23)

        if savefigures:
            plt.savefig(self.path_to_visual_outputs + '/selected_channels'+str(ch_index_list)+'.jpg')
        plt.close()

    def get_significant_components_state(self, potentially_significant, to_save = False):
            '''
            After the significant components were manually chosen,
            this function saves them into one variable stored withing an object of a class.
            :param potentially_significant: list(str)
            '''
            self.significant_components = {}

            for component in self.normalized_per_component_states:
                if component in potentially_significant:
                    self.significant_components[component] = self.normalized_per_component_states[component]

            if to_save:
                for i in self.significant_components:
                    f = self.path_to_results+'/significant_components_' + str(
                        i) + '.npy'
                    np.save(f, self.significant_components[i])




    def run_causal_inference(self, no_combination=3):
        '''
        The function runs the causal analysis on the extracted components. If the n.o. components of choice exceeds 3,
         all of the possible combinations by no_combination variable (recomennded = 3) are checked from the list.
        :param no_combination: int
        :return:
        '''
        if no_combination:
            possible_combinations = [p for p in multiset_permutations([i for i in self.significant_components.keys()],
                                                                      no_combination)]
        else:
            possible_combinations = [p for p in multiset_permutations([i for i in self.significant_components.keys()])]

        results = {}
        taus = [i for i in range(self.window_to_detect)]

        pval_min_threshold, pval_max_threshold = 0.08, 0.08

        for combination in possible_combinations:
            print('combination', combination)
            key_combination = tuple(combination)

            results[key_combination] = {}

            for tau in tqdm(taus):

                for t0 in range(self.window_to_detect * 2 - tau * 2):

                    x = self.significant_components[key_combination[0]][:, t0]
                    y = self.significant_components[key_combination[1]][:, t0 + tau * 2]
                    z = self.significant_components[key_combination[2]][:, t0 + tau]

                    data1 = pd.DataFrame({'x': x, 'y': y, 'covar': z})

                    pc = partial_corr(data1, 'x', 'y', 'covar')

                    correlation = corr(data1['x'], data1['y']).round(3)

                    if pc['p-val'][0] > pval_max_threshold:
                        if correlation['p-val'][0] < pval_min_threshold:
                            if tau in results[key_combination].keys():
                                results[key_combination][tau].append(t0)
                            else:

                                results[key_combination][tau] = [t0]

        self.per_combination_frequency = results


    def visualize_per_Tau_frequency(self, tau, savefigures = False):
        '''
        The function plots tick-plot for every causal event detected relative to an IED time for every possible combination of channels.
        :param tau: int
        :return:
        '''
        fig = plt.figure(figsize=(10, 10))
        timescale = list([i for i in range(-self.window_to_detect, self.window_to_detect)])
        plt.title("τ = " + str(tau))
        fixed_tau = tau
        Y_Tick_List = []
        Y_Tick_Label_List = []

        for indx, schema in enumerate(self.per_combination_frequency):

            plt.plot(timescale, list([indx * 10 for k in range(2*self.window_to_detect)]), label=schema[0] + ' -> ' + schema[2])
            Y_Tick_List.append(indx * 10)
            Y_Tick_Label_List.append(schema[0] + ' -> ' + schema[2])

            if fixed_tau in self.per_combination_frequency[schema].keys():
                all_markings = set([k - self.window_to_detect for k in self.per_combination_frequency[schema][fixed_tau]])
                plt.scatter(list(all_markings), list([indx * 10 for k in range(len(all_markings))]), marker="|")
            else:
                continue

        plt.yticks(ticks=Y_Tick_List, labels=Y_Tick_Label_List, rotation=25, fontsize=8)
        plt.axvline(x=0)

        if savefigures:
            plt.savefig(self.path_to_visual_outputs + '/per_tau_results/tau_'+str(tau)+'.jpg')
        plt.close()


In [16]:
dct = {
    'pat1': { },
    'pat2': { },
    'pat3': { },
    'pat4': {
            'wake': {'path_edf' : '/data/garkots/third_package/pat04_ON_wake/EEG_1277-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat04_ON_wake/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat4/wake',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat4/wake',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat4/wake',
                      'chosen_components' : ['ICA004', 'ICA003', 'ICA007']
                      },
             'sleep':{'path_edf' : '/data/garkots/third_package/pat04_ON_sleep/EEG_1278-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat04_ON_sleep/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat4/sleep',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat4/sleep',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat4/sleep',
                      'chosen_components' : ['ICA005', 'ICA003', 'ICA000']
                      }
             },
    'pat5': {
            'wake': {'path_edf' : '/data/garkots/third_package/pat05_OFF_wake/EEG_236-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat05_OFF_wake/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat5/wake',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat5/wake',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat5/wake',
                      'chosen_components' : ['ICA001', 'ICA006', 'ICA011']
                      },
             'sleep':{'path_edf' : '/data/garkots/third_package/pat05_OFF_sleep/EEG_1232-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat05_OFF_sleep/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat5/sleep',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat5/sleep',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat5/sleep',
                      'chosen_components' : ['ICA028', 'ICA018', 'ICA032']
                      }
             },
    'pat6' : {
            'wake': {'path_edf' : '/data/garkots/third_package/pat06_ON_wake/EEG_1231-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat06_ON_wake/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat6/wake',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat6/wake',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat6/wake',
                      'chosen_components' : ['ICA006','ICA016', 'ICA003', 'ICA008', 'ICA001', 'ICA025']
                      },
             'sleep':{'path_edf' : '/data/garkots/third_package/pat06_ON_sleep/EEG_1230-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat06_ON_sleep/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat6/sleep',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat6/sleep',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat6/sleep',
                      'chosen_components' : ['ICA007', 'ICA004', 'ICA024', 'ICA003']
                      }
             },
    
    'pat7' : {
            'wake': {'path_edf' : '/data/garkots/third_package/pat07_ON_wake/EEG_1185-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat07_ON_wake/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat7/wake',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat7/wake',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat7/wake',
                      'chosen_components' : ['ICA010', 'ICA023', 'ICA009', 'ICA006']
                      },
             'sleep':{'path_edf' : '/data/garkots/third_package/pat07_ON_sleep/pat07_EEG_1181-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat07_ON_sleep/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat7/sleep',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat7/sleep',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat7/sleep',
                      'chosen_components' : ['ICA005', 'ICA008', 'ICA013', 'ICA009', 'ICA006']
                      }
             },
    'pat8' : {
            'wake': {'path_edf' : '/data/garkots/third_package/pat08_ON_wake/EEG_1229-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat08_ON_wake/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat8/wake',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat8/wake',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat8/wake',
                      'chosen_components' : ['ICA000', 'ICA001', 'ICA002']
                      },
             'sleep':{'path_edf' : '/data/garkots/third_package/pat08_ON_sleep/EEG_1228-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat08_ON_sleep/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat8/sleep',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat8/sleep',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat8/sleep',
                      'chosen_components' : ['ICA000', 'ICA001', 'ICA002']
                      }
             },
    'pat9' : {
            'wake': {'path_edf' : '/data/garkots/third_package/pat09_ON_wake/EEG_1227-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat09_ON_wake/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat9/wake',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat9/wake',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat9/wake',
                      'chosen_components' : ['ICA006', 'ICA005', 'ICA000', 'ICA003']
                      },
             'sleep':{'path_edf' : '/data/garkots/third_package/pat09_ON_sleep/EEG_1225-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat09_ON_sleep/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat9/sleep',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat9/sleep',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat9/sleep',
                      'chosen_components' : ['ICA004', 'ICA006', 'ICA009', 'ICA012', 'ICA001']
                      }
             },
    'pat10' : {
            'wake': {'path_edf' : '/data/garkots/third_package/pat10_wake/EEG_1222-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat10_wake/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat10/wake',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat10/wake',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat10/wake',
                      'chosen_components' : ['ICA010',  'ICA005', 'ICA001']
                      },
             'sleep':{'path_edf' : '/data/garkots/third_package/pat10_sleep/EEG_1221-export.edf',
                      'path_ann' : '/data/garkots/third_package/pat10_sleep/',
                      'path_visual' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat10/sleep',
                      'path_results' : '/home/garkots/invasive_eeg/analysis/visuals/component_choice/experiment_window_100/pat10/sleep',
                      'path_ica' : '/home/garkots/invasive_eeg/analysis/matrices_ica/pat10/sleep',
                      'chosen_components' : ['ICA009', 'ICA005', 'ICA008']
                      }
             }
}

In [17]:
p, s = 'pat10', 'sleep'
patient = Patient(edf_file = dct[p][s]['path_edf'],
                 annotations =  dct[p][s]['path_ann'],
                 path_to_visual_outputs= dct[p][s]['path_visual'],
                 path_to_results= dct[p][s]['path_results'],
                 path_to_ica_matrix= dct[p][s]['path_ica'],
                 precomputed_ica = False,
                 ignore_channels = True,
                 window_length = 500,
                 chosen_components =dct[p][s]['chosen_components'],
                 control_experiment = False)

patient.quantify_annotations()
patient.run_ica()

CHOSEN COMPONENTS ['ICA009', 'ICA005', 'ICA008']
Extracting EDF parameters from /data/garkots/third_package/pat10_sleep/EEG_1221-export.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Channel  PAT10_SLEEP_EEG_1221 with  5728 markings
Total number of markings 5727
Fitting ICA to data using 138 channels (please be patient, this may take a while)


/tmp/ipykernel_355195/1683015212.py:96: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(self.raw)


Selecting by explained variance: 15 components
Fitting ICA took 114.2s.


In [18]:
patient.ica.mixing_matrix_.shape

(15, 15)

In [19]:
patient.ica.pca_components_.shape

(138, 138)

In [20]:
patient.ica.pca_explained_variance_.shape

(138,)

In [21]:
patient.ica.unmixing_matrix_.shape

(15, 15)

In [24]:
patient.ica.exclude

[]

In [26]:
patient.ica.pre_whitener_.shape

(138, 1)

In [27]:
patient.ica.n_components_

15

In [29]:
patient.ica.pca_explained_variance_

array([6.35659682e+01, 2.93753940e+01, 2.02985699e+01, 6.33974585e+00,
       3.84086799e+00, 1.42871199e+00, 1.26105145e+00, 1.08341247e+00,
       7.79646618e-01, 7.08221794e-01, 6.65750681e-01, 6.38511985e-01,
       5.52039764e-01, 5.02724868e-01, 4.75120061e-01, 4.26065876e-01,
       4.14207894e-01, 3.78865612e-01, 3.66165761e-01, 2.90479817e-01,
       2.39952968e-01, 2.34753776e-01, 2.27948795e-01, 2.04768283e-01,
       1.96495642e-01, 1.91397616e-01, 1.67668620e-01, 1.56090081e-01,
       1.38132300e-01, 1.35467474e-01, 1.26276425e-01, 1.20332245e-01,
       1.15267832e-01, 1.14663834e-01, 1.04086575e-01, 1.02564765e-01,
       9.97793611e-02, 9.74809383e-02, 9.59346083e-02, 8.74624119e-02,
       8.17931351e-02, 7.75649031e-02, 7.52917193e-02, 7.31771707e-02,
       7.22876062e-02, 6.68361155e-02, 6.41440737e-02, 6.20318126e-02,
       5.42463201e-02, 5.12809507e-02, 4.54233464e-02, 4.36609028e-02,
       4.05714191e-02, 3.93046707e-02, 3.65454432e-02, 3.59431811e-02,
      

In [42]:
g = patient.ica.get_components()

In [44]:
g[:,1].shape

(138,)

In [45]:
max(g[:,1]), min(g[:,1])

(2.5615784402949187, -0.23750664127930873)

In [46]:
import numpy as np
np.where(g[:,1] == 2.5615784402949187)

(array([126]),)

In [47]:

np.where(g[:,1] == -0.23750664127930873)

(array([137]),)

In [49]:
f = mne.io.read_raw_edf( dct[p][s]['path_edf'])

Extracting EDF parameters from /data/garkots/third_package/pat10_sleep/EEG_1221-export.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [51]:
f.ch_names[126]

'POL MRTB5'

In [55]:
a = np.array([9, 4, 4, 3, 3, 9, 0, 4, 6, 0])
ind = np.argpartition(a, -4)[-4:]

In [56]:
a[ind]

array([4, 9, 6, 9])

In [57]:
a = g[:,1]

In [58]:
ind = np.argpartition(a, -4)[-4:]

In [59]:
a[ind]

array([1.76709529, 1.99860536, 2.56157844, 2.31920247])

In [61]:
for i in ind:
    print(f.ch_names[i])

POL MRTB3
POL MRTB6
POL MRTB5
POL MRTB4
